In [125]:
# ! pip install torchtext==0.10.1
! pip install torchtext==0.6.0
! pip install datasets

NotImplementedError: ignored

In [126]:
from datasets import load_dataset

import torch
from torchtext import data

In [127]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [128]:
MODEL_CONFIG = "CONCAT"
print("Using MODEL_CONFIG", MODEL_CONFIG)

Using MODEL_CONFIG CONCAT


In [129]:
PROJECT_ROOT = F"/content/gdrive/My Drive/nlp_project_task_1/"

In [130]:
SEED = 42
MAX_VOCAB_SIZE = 25_000

In [131]:
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [132]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [133]:
faithdial_dataset = load_dataset("McGill-NLP/FaithDial")

  0%|          | 0/7 [00:00<?, ?it/s]

In [134]:
faithdial_dataset.keys()

dict_keys(['test', 'test_random_split', 'test_topic_split', 'train', 'validation', 'valid_random_split', 'valid_topic_split'])

In [135]:
faithdial_dataset["train"][0]

{'dialog_idx': 0,
 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?',
 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'history': ['Can you imagine the world without internet access?'],
 'knowledge': 'Internet access was once rare, but has grown rapidly.',
 'BEGIN': ['Hallucination'],
 'VRM': ['Disclosure', 'Ack.']}

In [136]:
def critic_preprocess(dataset):
    """
    Data items transformed into (knowledge, response, is_hallucination)
    """
    new_dataset = []
    for d in dataset:
        # original response
        if d["original_response"] != None:
            new_dataset.append({
                "knowledge": d["knowledge"],
                "response": d["original_response"],
                "hallucination": "yes" if "Hallucination" in d["BEGIN"] else "no",
                "history": " ".join(d["history"]),
                "all": " ".join(d["history"]) + " <eos> " + d["knowledge"] + " <eos> " + d["original_response"]
            })

        # new responses always aren't hallucinations
        new_dataset.append({"knowledge": d["knowledge"],
                            "response": d["response"],
                            "hallucination": "no",
                            "history": " ".join(d["history"]),
                            "all": " ".join(d["history"]) + " <eos> " + d["knowledge"] + " <eos> " + d["response"]
        })
    return new_dataset

In [137]:
import json

def dump_as_json(dataset, filename):
    """
    Takes a list of dicts and dumps it as a json file that torchtext can parse.
    """
    with open(filename, "w") as file:
        for d in dataset:
            file.write(json.dumps(d))
            file.write("\n")


In [138]:
KNOWLEDGE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
RESPONSE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
HISTORY = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
LABEL = data.LabelField(dtype=torch.float)

ALL = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)

In [139]:
dump_as_json(critic_preprocess(faithdial_dataset["test"]), PROJECT_ROOT + "data/faithdial_dataset_test.json")
dump_as_json(critic_preprocess(faithdial_dataset["train"]), PROJECT_ROOT + "data/faithdial_dataset_train.json")
dump_as_json(critic_preprocess(faithdial_dataset["validation"]), PROJECT_ROOT + "data/faithdial_dataset_validation.json")

In [140]:
# fields = {"knowledge": ("k", KNOWLEDGE), "response": ("r", RESPONSE), "hallucination": ("l", LABEL), "history": ("h", HISTORY)}
fields = {"all": ("a", ALL), "hallucination": ("l", LABEL)}

dataset = data.TabularDataset.splits(path=PROJECT_ROOT + "data",
                                     train="faithdial_dataset_train.json",
                                     validation="faithdial_dataset_validation.json",
                                     test="faithdial_dataset_test.json",
                                     format="json",
                                     fields=fields)


In [141]:
train_data, valid_data, test_data = dataset

In [142]:
train_data[0]

In [143]:
vars(train_data.examples[0])

{'a': ['Can',
  'you',
  'imagine',
  'the',
  'world',
  'without',
  'internet',
  'access',
  '?',
  '<',
  'eos',
  '>',
  'Internet',
  'access',
  'was',
  'once',
  'rare',
  ',',
  'but',
  'has',
  'grown',
  'rapidly',
  '.',
  '<',
  'eos',
  '>',
  'No',
  'I',
  'could',
  'not',
  '!',
  'I',
  'could',
  "n't",
  'imagine',
  'living',
  'when',
  'internet',
  'access',
  'was',
  'rare',
  'and',
  'very',
  'few',
  'people',
  'had',
  'it',
  '!'],
 'l': 'yes'}

In [144]:
KNOWLEDGE.build_vocab(train_data,
                      max_size=MAX_VOCAB_SIZE,
                      vectors = "fasttext.simple.300d",
                      unk_init = torch.Tensor.normal_)
RESPONSE.build_vocab(train_data,
                     max_size=MAX_VOCAB_SIZE,
                     vectors = "fasttext.simple.300d",
                     unk_init = torch.Tensor.normal_)
HISTORY.build_vocab(train_data,
                    max_size=MAX_VOCAB_SIZE,
                    vectors = "fasttext.simple.300d",
                    unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

ALL.build_vocab(train_data,
                specials=["<eos>"],
                max_size=MAX_VOCAB_SIZE,
                vectors = "fasttext.simple.300d",
                unk_init = torch.Tensor.normal_)


In [145]:
print(f"Unique tokens in KNOWLEDGE vocabulary: {len(KNOWLEDGE.vocab)}")
print(f"Unique tokens in RESPONSE vocabulary: {len(RESPONSE.vocab)}")
print(f"Unique tokens in HISTORY vocabulary: {len(HISTORY.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(f"Unique tokens in ALL vocabulary: {len(ALL.vocab)}")

Unique tokens in KNOWLEDGE vocabulary: 2
Unique tokens in RESPONSE vocabulary: 2
Unique tokens in HISTORY vocabulary: 2
Unique tokens in LABEL vocabulary: 2
Unique tokens in ALL vocabulary: 25003


In [146]:
print(KNOWLEDGE.vocab.freqs.most_common(20))
print(RESPONSE.vocab.freqs.most_common(20))
print(HISTORY.vocab.freqs.most_common(20))
print(LABEL.vocab.freqs.most_common(20))

print(ALL.vocab.freqs.most_common(20))

[]
[]
[]
[('no', 20474), ('yes', 13507)]
[(',', 204210), ('.', 194505), ('the', 136207), ('I', 132588), ('a', 94289), ('of', 85297), ('and', 79933), ('?', 77908), ('is', 72173), ('to', 68896), ('>', 67993), ('<', 67966), ('eos', 67962), ('in', 64071), ('you', 62139), ('that', 58011), ('it', 46102), ('know', 44239), ("'s", 32481), ('are', 32411)]


In [147]:
print(KNOWLEDGE.vocab.itos[:10])
print(RESPONSE.vocab.itos[:10])
print(HISTORY.vocab.itos[:10])
print(LABEL.vocab.itos[:10])

print(ALL.vocab.itos[:10])

['<unk>', '<pad>']
['<unk>', '<pad>']
['<unk>', '<pad>']
['no', 'yes']
['<unk>', '<pad>', '<eos>', ',', '.', 'the', 'I', 'a', 'of', 'and']


In [148]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    # sort_key = lambda x: x.r,
    sort_key = lambda x: x.a,
    device = device)

In [149]:
from torch import nn

class LSTM(nn.Module):
    def __init__(self, response_vocab_size, knowledge_vocab_size, history_vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, response_pad_idx, knowledge_pad_idx, history_pad_idx):

        super().__init__()

        # Initialize Embedding Layer
        self.response_embedding = nn.Embedding(num_embeddings=response_vocab_size,
                                               embedding_dim=embedding_dim,
                                               padding_idx=response_pad_idx)

        self.knowledge_embedding = nn.Embedding(num_embeddings=knowledge_vocab_size,
                                                embedding_dim=embedding_dim,
                                                padding_idx=knowledge_pad_idx)
        
        self.history_embedding = nn.Embedding(num_embeddings=history_vocab_size,
                                              embedding_dim=embedding_dim,
                                              padding_idx=history_pad_idx)

        # Initialize LSTM layer
        self.response_lstm = nn.LSTM(input_size=embedding_dim,
                                     hidden_size=hidden_dim,
                                     num_layers=n_layers,
                                     bidirectional=bidirectional)

        self.knowledge_lstm = nn.LSTM(input_size=embedding_dim,
                                      hidden_size=hidden_dim,
                                      num_layers=n_layers,
                                      bidirectional=bidirectional)
        
        self.history_lstm = nn.LSTM(input_size=embedding_dim,
                                    hidden_size=hidden_dim,
                                    num_layers=n_layers,
                                    bidirectional=bidirectional)

        # Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=3*2*hidden_dim,
                            out_features=output_dim)

        # Initialize Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, response, response_lengths, knowledge, knowledge_lengths, history, history_lengths):
        # Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        x_r = self.response_embedding(response)
        x_r = self.dropout(x_r)

        x_k = self.knowledge_embedding(knowledge)
        x_k = self.dropout(x_k)

        x_h = self.history_embedding(history)
        x_h = self.dropout(x_h)

        # Run the LSTM along the sentences of length sent_len.
        output_r, (hidden_r, cell_r) = self.response_lstm(x_r)
        output_k, (hidden_k, cell_k) = self.knowledge_lstm(x_k)
        output_h, (hidden_h, cell_h) = self.history_lstm(x_h)

        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        hidden_r = torch.cat((hidden_r[-2,:,:], hidden_r[-1,:,:]), -1)
        hidden_k = torch.cat((hidden_k[-2,:,:], hidden_k[-1,:,:]), -1)
        hidden_h = torch.cat((hidden_h[-2,:,:], hidden_h[-1,:,:]), -1)
        hidden = torch.cat((hidden_r, hidden_k, hidden_h), -1)
        hidden = self.dropout(hidden)

        return self.fc(hidden)

In [150]:
from torch import nn

class ConcatLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        # Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                               embedding_dim=embedding_dim,
                                               padding_idx=pad_idx)

        # Initialize LSTM layer
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            bidirectional=bidirectional)

        # Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=2*hidden_dim,
                            out_features=output_dim)

        # Initialize Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, all, all_lengths):
        # Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        x = self.embedding(all)
        x = self.dropout(x)

        # Run the LSTM along the sentences of length sent_len.
        output, (hidden, cell) = self.lstm(x)

        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), -1)
        hidden = self.dropout(hidden)

        return self.fc(hidden)

In [151]:
RESPONSE_INPUT_DIM = len(RESPONSE.vocab)
KNOWLEDGE_INPUT_DIM = len(KNOWLEDGE.vocab)
HISTORY_INPUT_DIM = len(HISTORY.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
RESPONSE_PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]
KNOWLEDGE_PAD_IDX = KNOWLEDGE.vocab.stoi[KNOWLEDGE.pad_token]
HISTORY_PAD_IDX = HISTORY.vocab.stoi[HISTORY.pad_token]


# model = LSTM(RESPONSE_INPUT_DIM,
#              KNOWLEDGE_INPUT_DIM,
#              HISTORY_INPUT_DIM,
#              EMBEDDING_DIM,
#              HIDDEN_DIM,
#              OUTPUT_DIM,
#              N_LAYERS,
#              BIDIRECTIONAL,
#              DROPOUT,
#              RESPONSE_PAD_IDX,
#              KNOWLEDGE_PAD_IDX,
#              HISTORY_PAD_IDX)


ALL_PAD_IDX = ALL.vocab.stoi[ALL.pad_token]

model = ConcatLSTM(len(ALL.vocab), EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, ALL_PAD_IDX)

In [152]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,221,157 trainable parameters


In [153]:
print(RESPONSE.vocab.vectors.shape)
print(KNOWLEDGE.vocab.vectors.shape)
print(HISTORY.vocab.vectors.shape)

torch.Size([2, 300])
torch.Size([2, 300])
torch.Size([2, 300])


In [154]:
# model.response_embedding.weight.data.copy_(RESPONSE.vocab.vectors)
# model.knowledge_embedding.weight.data.copy_(KNOWLEDGE.vocab.vectors)
# model.history_embedding.weight.data.copy_(HISTORY.vocab.vectors)

model.embedding.weight.data.copy_(ALL.vocab.vectors)

tensor([[ 1.2538,  0.0902,  1.9940,  ...,  0.3880, -0.0469, -0.6670],
        [ 0.6487, -1.5245, -0.4629,  ..., -0.0669, -0.6547,  1.0335],
        [-2.1418,  0.7724, -0.6359,  ...,  0.6256, -0.6219, -1.0873],
        ...,
        [ 0.2151,  0.4464,  1.8746,  ..., -0.5294,  0.3751, -0.6135],
        [-1.2156,  0.7609, -1.9293,  ..., -0.3884, -0.3926, -0.1387],
        [ 0.8763, -1.6594,  1.9265,  ..., -1.4424,  0.7236,  0.6938]])

In [155]:
UNK_IDX_R = RESPONSE.vocab.stoi[RESPONSE.unk_token]
UNK_IDX_K = RESPONSE.vocab.stoi[KNOWLEDGE.unk_token]
UNK_IDX_H = RESPONSE.vocab.stoi[HISTORY.unk_token]

UNK_IDX_A = ALL.vocab.stoi[ALL.unk_token]

# model.response_embedding.weight.data[UNK_IDX_R] = torch.zeros(EMBEDDING_DIM)
# model.response_embedding.weight.data[RESPONSE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# model.knowledge_embedding.weight.data[UNK_IDX_K] = torch.zeros(EMBEDDING_DIM)
# model.knowledge_embedding.weight.data[KNOWLEDGE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# model.history_embedding.weight.data[UNK_IDX_H] = torch.zeros(EMBEDDING_DIM)
# model.history_embedding.weight.data[HISTORY_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.embedding.weight.data[UNK_IDX_A] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[ALL_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# print(model.response_embedding.weight.data)
# print(model.knowledge_embedding.weight.data)
# print(model.history_embedding.weight.data)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-2.1418,  0.7724, -0.6359,  ...,  0.6256, -0.6219, -1.0873],
        ...,
        [ 0.2151,  0.4464,  1.8746,  ..., -0.5294,  0.3751, -0.6135],
        [-1.2156,  0.7609, -1.9293,  ..., -0.3884, -0.3926, -0.1387],
        [ 0.8763, -1.6594,  1.9265,  ..., -1.4424,  0.7236,  0.6938]])


In [156]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [157]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [158]:
from sklearn.metrics import f1_score


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


def binary_f1(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    # f1 = f1_score(y.detach().cpu(), rounded_preds.detatch().cpu(), average="macro")
    f1 = f1_score(y.cpu(), rounded_preds.cpu(), average="macro")

    return f1


In [159]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        # response, response_lengths = batch.r
        # knowledge, knowledge_lengths = batch.k
        # history, history_lengths = batch.h

        # predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)



        all, lengths = batch.a
        predictions = model(all, lengths).squeeze(1)
        

        

        loss = criterion(predictions, batch.l)

        acc = binary_accuracy(predictions, batch.l)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [160]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:
            # response, response_lengths = batch.r
            # knowledge, knowledge_lengths = batch.k
            # history, history_lengths = batch.h

            # predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)


            all, lengths = batch.a
            predictions = model(all, lengths).squeeze(1)


            loss = criterion(predictions, batch.l)
            acc = binary_accuracy(predictions, batch.l)
            f1 = binary_f1(predictions, batch.l)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [161]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [162]:
N_EPOCHS = 5
path = PROJECT_ROOT + "/" + MODEL_CONFIG + "_model.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. F1: {valid_f1:.3f}')

Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.674 | Train Acc: 60.29% |
	 Val. Loss: 0.673 |  Val. Acc: 60.13% | Val. F1: 0.379
Epoch: 02 | Epoch Time: 1m 7s
	Train Loss: 0.675 | Train Acc: 60.09% |
	 Val. Loss: 0.672 |  Val. Acc: 60.14% | Val. F1: 0.378
Epoch: 03 | Epoch Time: 1m 8s
	Train Loss: 0.674 | Train Acc: 60.22% |
	 Val. Loss: 0.672 |  Val. Acc: 60.24% | Val. F1: 0.385
Epoch: 04 | Epoch Time: 1m 8s
	Train Loss: 0.671 | Train Acc: 60.42% |
	 Val. Loss: 0.672 |  Val. Acc: 60.17% | Val. F1: 0.378
Epoch: 05 | Epoch Time: 1m 9s
	Train Loss: 0.675 | Train Acc: 59.90% |
	 Val. Loss: 0.673 |  Val. Acc: 60.14% | Val. F1: 0.378


In [163]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1:.2f}')

Test Loss: 0.670 | Test Acc: 60.59% | Test F1: 0.39


In [164]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_hallucination(model, knowledge, response):
    model.eval()

    tokenized_r = [tok.text for tok in nlp.tokenizer(response)]
    indexed_r = [RESPONSE.vocab.stoi[t] for t in tokenized_r]
    length_r = [len(indexed_r)]
    tensor_r = torch.LongTensor(indexed_r).to(device)
    tensor_r = tensor_r.unsqueeze(1)
    length_tensor_r = torch.LongTensor(length_r)

    tokenized_k = [tok.text for tok in nlp.tokenizer(knowledge)]
    indexed_k = [KNOWLEDGE.vocab.stoi[t] for t in tokenized_k]
    length_k = [len(indexed_k)]
    tensor_k = torch.LongTensor(indexed_k).to(device)
    tensor_k = tensor_k.unsqueeze(1)
    length_tensor_k = torch.LongTensor(length_k)

    prediction = torch.sigmoid(model(tensor_r, length_tensor_r, tensor_k, length_tensor_k))

    return prediction.item()


In [165]:
predict_hallucination(model, "", "I love dogs")

TypeError: ignored

In [ ]:
predict_hallucination(model, "", "Dogs are animals.")

In [ ]:
predict_hallucination(model, "", "I was walking my dog last week.")

In [ ]:
predict_hallucination(model, "", "Dogs need to be walked daily.")

In [ ]:
test_data[2].r

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is a candy supplier.")

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is my favorite great brand of candy.")

In [ ]:
print(test_data[2].h)